In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from pathlib import Path

plt.style.use("seaborn-v0_8-darkgrid")

# Localisation du fichier CSV
current_dir = Path().resolve()
project_root = current_dir.parent
data_path = project_root / "Data" / "Dataset of weighing station temperature measurements.csv"

df = pd.read_csv(data_path, sep=";")

# Conversion de la colonne Time en datetime et mise en index
df["Time"] = pd.to_datetime(df["Time"])
df = df.set_index("Time").sort_index()

df.head()


In [ ]:
# Colonnes de température à l'intérieur du puits
low_cols = [c for c in df.columns if "Low" in c]
mid_cols = [c for c in df.columns if "Mid" in c]
top_cols = [c for c in df.columns if "Top" in c]

# Fonction pour trier par numéro de sonde S1, S2, ...
def sort_by_sensor(col_list):
    return sorted(col_list, key=lambda x: int(x.split("-S")[1]))

low_cols = sort_by_sensor(low_cols)
mid_cols = sort_by_sensor(mid_cols)
top_cols = sort_by_sensor(top_cols)

# print("Nb sondes LOW :", len(low_cols))
# print("Exemple LOW :", low_cols[:5])


In [ ]:
zones_dict = {
    "P1": [1, 2, 3, 4, 5],
    "P2": [6, 7, 8, 9],
    "P3": [10, 11, 12, 13, 14],
    "P4": [15, 16, 17, 18, 19],
    "P5": [20, 21, 22, 23, 24],
    "P6": [25, 26, 27, 28, 29],
}

# Température moyenne sous chaque plateau (LOW + MID + TOP)

plateaux_temp = {}   # dict: "P1" série temporelle

for zone_name, sondes in zones_dict.items():
    # Moyenne horizontale sur les sondes de ce plateau pour chaque hauteur
    T_low = df[[low_cols[s-1] for s in sondes]].mean(axis=1)
    T_mid = df[[mid_cols[s-1] for s in sondes]].mean(axis=1)
    T_top = df[[top_cols[s-1] for s in sondes]].mean(axis=1)
    
    # Moyenne verticale (LOW + MID + TOP)
    T_mean = pd.concat([T_low, T_mid, T_top], axis=1).mean(axis=1)
    
    plateaux_temp[zone_name] = T_mean

# DataFrame final : une colonne par plateau
df_plateaux = pd.DataFrame(plateaux_temp)

# smoothing temporel (30 minutes)
df_plateaux_30min = df_plateaux.resample("30min").mean()

df_plateaux_30min.head()


In [ ]:
#graph evolution temperature moyenne sous chaque plateau
date_split = dt.datetime(2024, 1, 26)

for plateau in df_plateaux_30min.columns:
    plt.figure(figsize=(12, 4))
    plt.plot(df_plateaux_30min.index,
             df_plateaux_30min[plateau],
             label=f"Température moyenne sous {plateau}",
             linewidth=1.2)

    # Ligne verticale pour la réparation du P6 (optionnel)
    plt.axvline(date_split, color="k", linestyle="--", alpha=0.6,
                label="26 janv (réparation)")

    plt.xlabel("Temps")
    plt.ylabel("Température moyenne [°C]")
    plt.title(f"Évolution de la température moyenne sous {plateau}")
    plt.legend()
    plt.tight_layout()

    # Format de l’axe temporel
    plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    plt.gcf().autofmt_xdate()

    plt.show()


In [ ]:
# Profil journalier moyen pour chaque plateau
# Comparaison : toute la période vs après la réparation (26 janvier)

date_split = dt.datetime(2024, 1, 26)

for p in df_plateaux_30min.columns:
    # Série complète pour ce plateau
    serie = df_plateaux_30min[p].dropna()

    # Profil journalier moyen sur toute la période
    profil_journee_total = serie.groupby(serie.index.hour).mean()

    # Profil journalier moyen après le 26 janvier
    serie_apres = serie[serie.index >= date_split]
    profil_journee_apres = serie_apres.groupby(serie_apres.index.hour).mean()

    plt.figure(figsize=(6, 4))
    plt.plot(
        profil_journee_total.index,
        profil_journee_total.values,
        marker="o",
        linewidth=1.3,
        label="Toute la période",
    )
    plt.plot(
        profil_journee_apres.index,
        profil_journee_apres.values,
        marker="s",
        linewidth=1.3,
        linestyle="--",
        label="Après 26 janv.",
    )

    plt.xlabel("Heure de la journée")
    plt.ylabel(f"Température moyenne sous {p} [°C]")
    plt.title(f"Profil journalier moyen – {p}")
    plt.grid(True, alpha=0.4)
    plt.xticks(range(0, 24, 2))  # ticks toutes les 2 h
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
date_split = dt.datetime(2024, 1, 26)

stats_global = {}
stats_avant = {}
stats_apres = {}

for p in df_plateaux_30min.columns:
    # Série complète pour ce plateau (sans NaN)
    s = df_plateaux_30min[p].dropna()

    # Si aucune donnée, on passe
    if s.empty:
        continue

    # --- Statistiques sur toute la période ---
    stats_global[p] = {
        "n_global": len(s),
        "mean_global": s.mean(),
        "std_global":  s.std(),
        "min_global":  s.min(),
        "max_global":  s.max(),
    }

    # --- Séparation avant / après 26 janvier ---
    s_av = s[s.index < date_split]
    s_ap = s[s.index >= date_split]

    # Avant 26 janvier
    stats_avant[p] = {
        "n_avant":   len(s_av),
        "mean_avant": s_av.mean() if not s_av.empty else float("nan"),
        "min_avant":  s_av.min()  if not s_av.empty else float("nan"),
        "max_avant":  s_av.max()  if not s_av.empty else float("nan"),
    }

    # Après 26 janvier
    stats_apres[p] = {
        "n_apres":   len(s_ap),
        "mean_apres": s_ap.mean() if not s_ap.empty else float("nan"),
        "min_apres":  s_ap.min()  if not s_ap.empty else float("nan"),
        "max_apres":  s_ap.max()  if not s_ap.empty else float("nan"),
    }

# Création des DataFrames individuels
df_stats_globale = pd.DataFrame(stats_global).T
df_stats_avant   = pd.DataFrame(stats_avant).T
df_stats_apres   = pd.DataFrame(stats_apres).T

# Fusion en un seul tableau
df_stats = pd.concat(
    [df_stats_globale, df_stats_avant, df_stats_apres],
    axis=1
)

# Ajout des deltas après - avant (utile pour le commentaire dans le rapport)
df_stats["delta_mean_apres_avant"] = df_stats["mean_apres"] - df_stats["mean_avant"]
df_stats["delta_min_apres_avant"]  = df_stats["min_apres"]  - df_stats["min_avant"]
df_stats["delta_max_apres_avant"]  = df_stats["max_apres"]  - df_stats["max_avant"]

# Arrondi pour un affichage propre
df_stats_rounded = df_stats.round(2)

print("Tableau synthèse des statistiques par plateau (°C)")
display(df_stats_rounded)
